# Planet Simulator with Tensorflow Pipeline

Author: Craig Boger
06/07/2020

This script takes the model prototyping and learning of v1.01 and tries to expand upon it in 2 key areas.

1) Perform data normalization and processing in TF data libraries.
2) Take predictions in normalized form and output them in their unormalized form for use in simulation.

## Straight Up Just Stealing Someone's Code and Trying to Run It

Credit to benrules2: https://gist.github.com/benrules2/220d56ea6fe9a85a4d762128b11adfba

In [1]:
import math
import random
%matplotlib widget
import matplotlib.pyplot as plot
from mpl_toolkits.mplot3d import Axes3D

class point:
    def __init__(self, x,y,z):
        self.x = x
        self.y = y
        self.z = z

class body:
    def __init__(self, location, mass, velocity, name = ""):
        self.location = location
        self.mass = mass
        self.velocity = velocity
        self.name = name

def calculate_single_body_acceleration(bodies, body_index):
    G_const = 6.67408e-11 #m3 kg-1 s-2
    acceleration = point(0,0,0)
    target_body = bodies[body_index]
    for index, external_body in enumerate(bodies):
        if index != body_index:
            r = (target_body.location.x - external_body.location.x)**2 + (target_body.location.y - external_body.location.y)**2 + (target_body.location.z - external_body.location.z)**2
            r = math.sqrt(r)
            tmp = G_const * external_body.mass / r**3
            acceleration.x += tmp * (external_body.location.x - target_body.location.x)
            acceleration.y += tmp * (external_body.location.y - target_body.location.y)
            acceleration.z += tmp * (external_body.location.z - target_body.location.z)

    return acceleration

def compute_velocity(bodies, time_step = 1):
    for body_index, target_body in enumerate(bodies):
        acceleration = calculate_single_body_acceleration(bodies, body_index)

        target_body.velocity.x += acceleration.x * time_step
        target_body.velocity.y += acceleration.y * time_step
        target_body.velocity.z += acceleration.z * time_step 


def update_location(bodies, time_step = 1):
    for target_body in bodies:
        target_body.location.x += target_body.velocity.x * time_step
        target_body.location.y += target_body.velocity.y * time_step
        target_body.location.z += target_body.velocity.z * time_step

def compute_gravity_step(bodies, time_step = 1):
    compute_velocity(bodies, time_step = time_step)
    update_location(bodies, time_step = time_step)

def plot_output(bodies, outfile = None):
    fig = plot.figure()
    colours = ['r','b','g','y','m','c']
    ax = fig.add_subplot(1,1,1, projection='3d')
    max_range = 0
    for current_body in bodies: 
        max_dim = max(max(current_body["x"]),max(current_body["y"]),max(current_body["z"]))
        if max_dim > max_range:
            max_range = max_dim
        ax.plot(current_body["x"], current_body["y"], current_body["z"], c = random.choice(colours), label = current_body["name"])        
    
    ax.set_xlim([-max_range,max_range])    
    ax.set_ylim([-max_range,max_range])
    ax.set_zlim([-max_range,max_range])
    ax.legend()        

    if outfile:
        plot.savefig(outfile)
    else:
        plot.show()

def run_simulation(bodies, names = None, time_step = 1, number_of_steps = 10000, report_freq = 100):

    #create output container for each body
    body_locations_hist = []
    for current_body in bodies:
        body_locations_hist.append({"x":[], "y":[], "z":[], "name":current_body.name})
        
    for i in range(1,number_of_steps):
        compute_gravity_step(bodies, time_step = 1000)            
        
        if i % report_freq == 0:
            for index, body_location in enumerate(body_locations_hist):
                body_location["x"].append(bodies[index].location.x)
                body_location["y"].append(bodies[index].location.y)           
                body_location["z"].append(bodies[index].location.z)       

    return body_locations_hist        
            
#planet data (location (m), mass (kg), velocity (m/s)
sun = {"location":point(0,0,0), "mass":2e30, "velocity":point(0,0,0)}
mercury = {"location":point(0,5.7e10,0), "mass":3.285e23, "velocity":point(47000,0,0)}
venus = {"location":point(0,1.1e11,0), "mass":4.8e24, "velocity":point(35000,0,0)}
earth = {"location":point(0,1.5e11,0), "mass":6e24, "velocity":point(30000,0,0)}
mars = {"location":point(0,2.2e11,0), "mass":2.4e24, "velocity":point(24000,0,0)}
jupiter = {"location":point(0,7.7e11,0), "mass":1e28, "velocity":point(13000,0,0)}
saturn = {"location":point(0,1.4e12,0), "mass":5.7e26, "velocity":point(9000,0,0)}
uranus = {"location":point(0,2.8e12,0), "mass":8.7e25, "velocity":point(6835,0,0)}
neptune = {"location":point(0,4.5e12,0), "mass":1e26, "velocity":point(5477,0,0)}
pluto = {"location":point(0,3.7e12,0), "mass":1.3e22, "velocity":point(4748,0,0)}
# TODO: Add random sattellite here.
satellite_1 = {"location":point(1e5,3.7e5,0), "mass":1.7e1, "velocity":point(4748,0,0)}

if __name__ == "__main__":

    #build list of planets in the simulation, or create your own
    bodies = [
        body( location = sun["location"], mass = sun["mass"], velocity = sun["velocity"], name = "sun"),
        body( location = earth["location"], mass = earth["mass"], velocity = earth["velocity"], name = "earth"),
        body( location = mars["location"], mass = mars["mass"], velocity = mars["velocity"], name = "mars"),
        body( location = venus["location"], mass = venus["mass"], velocity = venus["velocity"], name = "venus"),
        body( location = mercury["location"], mass = mercury["mass"], velocity = mercury["velocity"], name = "mercury"),
        body( location = jupiter["location"], mass = jupiter["mass"], velocity = jupiter["velocity"], name = "jupiter"),
        body( location = saturn["location"], mass = saturn["mass"], velocity = saturn["velocity"], name = "saturn"),
        
        #body( location = satellite_1["location"], mass = satellite_1["mass"], velocity = satellite_1["velocity"], name = "sattellite_1")
        ]
    
    # Original defaults of simulation
    # motions = run_simulation(bodies, time_step = 100, number_of_steps = 80000, report_freq = 1000)
    # Try messing with report frequency to get more data.
    motions = run_simulation(bodies, time_step = 100, number_of_steps = 300000, report_freq = 100)
    plot_output(motions, outfile = 'orbits.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Take motions data from the above simulation and convert it to a Pandas dataframe.  The "motions" output is a list of python dictionaries that can be converted into a dataframe and then manipulated.

In [2]:
import pandas as pd
import numpy as np

motions_df = pd.DataFrame(motions)
motions_df.head(100)

,x,y,z,name
0,"[6.172247210875407, 49.37608388489302, 166.613...","[6056.974666438165, 24107.571980251058, 54150....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sun
1,"[2999802268.7576785, 5998418130.158717, 899466...","[149970049800.53976, 149880804272.02502, 14973...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",earth
2,"[2399949856.0043564, 4799598821.688435, 719864...","[219986083529.99896, 219944611139.58228, 21987...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mars
3,"[3499415066.461402, 6995320910.112684, 1048421...","[109944304437.46532, 109778376868.05923, 10950...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",venus
4,"[4694355206.034602, 9354859805.633783, 1394777...","[56792631335.43017, 56175843268.08082, 5515329...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mercury
5,"[1299999366.480654, 2599994931.4665866, 389998...","[769998863548.677, 769995476701.667, 769989839...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",jupiter
6,"[899999928.8208524, 1799999430.5240006, 269999...","[1399999647585.8357, 1399998597321.9666, 13999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",saturn


In [3]:
# Trying to separate out each row of list or dataframe into its own dataframe.
# Will later put these dataframes back together into 1 large dataframe.

motions_df_list = []
for body in motions:
    motions_df_list.append(pd.DataFrame(body))

In [4]:
motions_df_list[3]

,x,y,z,name
0,3.499415e+09,1.099443e+11,0.0,venus
1,6.995321e+09,1.097784e+11,0.0,venus
2,1.048421e+10,1.095024e+11,0.0,venus
3,1.396259e+10,1.091166e+11,0.0,venus
4,1.742697e+10,1.086215e+11,0.0,venus
...,...,...,...,...
2994,-5.289650e+10,8.213631e+10,0.0,venus
2995,-5.030464e+10,8.447291e+10,0.0,venus
2996,-4.763994e+10,8.672745e+10,0.0,venus
2997,-4.490500e+10,8.889761e+10,0.0,venus


In [5]:
# Combine the dataframes into a single, large dataframe.
# Can later choose a planet to be the target we train to predict.
complete_motion_df = None

for body in motions_df_list:
    # Append name of body to each column and remove the name column
    body_name = body.loc[0, "name"]
    body.columns = [body_name + "_x",
                    body_name + "_y",
                    body_name + "_z",
                    "name"]
    # Add current body to the complete dataframe.
    complete_motion_df = pd.concat([complete_motion_df, body.iloc[:, 0:3]], axis=1)

complete_motion_df.head(100)

,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,mars_x,mars_y,mars_z,venus_x,...,venus_z,mercury_x,mercury_y,mercury_z,jupiter_x,jupiter_y,jupiter_z,saturn_x,saturn_y,saturn_z
0,6.172247e+00,6.056975e+03,0.0,2.999802e+09,1.499700e+11,0.0,2.399950e+09,2.199861e+11,0.0,3.499415e+09,...,0.0,4.694355e+09,5.679263e+10,0.0,1.299999e+09,7.699989e+11,0.0,8.999999e+08,1.400000e+12,0.0
1,4.937608e+01,2.410757e+04,0.0,5.998418e+09,1.498808e+11,0.0,4.799599e+09,2.199446e+11,0.0,6.995321e+09,...,0.0,9.354860e+09,5.617584e+10,0.0,2.599995e+09,7.699955e+11,0.0,1.799999e+09,1.399999e+12,0.0
2,1.666139e+02,5.415063e+04,0.0,8.994662e+09,1.497323e+11,0.0,7.198646e+09,2.198756e+11,0.0,1.048421e+10,...,0.0,1.394778e+10,5.515330e+10,0.0,3.899983e+09,7.699898e+11,0.0,2.699998e+09,1.399997e+12,0.0
3,3.948280e+02,9.618418e+04,0.0,1.198735e+10,1.495246e+11,0.0,9.596791e+09,2.197790e+11,0.0,1.396259e+10,...,0.0,1.843961e+10,5.373113e+10,0.0,5.199959e+09,7.699820e+11,0.0,3.599995e+09,1.399994e+12,0.0
4,7.708690e+02,1.502055e+05,0.0,1.497529e+10,1.492578e+11,0.0,1.199373e+10,2.196549e+11,0.0,1.742697e+10,...,0.0,2.279721e+10,5.191794e+10,0.0,6.499921e+09,7.699718e+11,0.0,4.499991e+09,1.399991e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.138605e+06,5.394756e+07,0.0,1.441446e+11,-4.894462e+10,0.0,1.875072e+11,1.033965e+11,0.0,1.578892e+10,...,0.0,3.050288e+10,-4.133032e+10,0.0,1.242400e+11,7.596504e+11,0.0,8.633698e+10,1.396785e+12,0.0
96,4.259034e+06,5.506067e+07,0.0,1.431961e+11,-5.174263e+10,0.0,1.885864e+11,1.011770e+11,0.0,1.238219e+10,...,0.0,2.606635e+10,-4.407853e+10,0.0,1.255223e+11,7.594341e+11,0.0,8.723499e+10,1.396718e+12,0.0
97,4.381576e+06,5.618472e+07,0.0,1.421936e+11,-5.452104e+10,0.0,1.896398e+11,9.894374e+10,0.0,8.963735e+09,...,0.0,2.137176e+10,-4.638960e+10,0.0,1.268043e+11,7.592156e+11,0.0,8.813296e+10,1.396650e+12,0.0
98,4.506234e+06,5.731969e+07,0.0,1.411373e+11,-5.727882e+10,0.0,1.906674e+11,9.669701e+10,0.0,5.536788e+09,...,0.0,1.646399e+10,-4.823713e+10,0.0,1.280859e+11,7.589949e+11,0.0,8.903089e+10,1.396581e+12,0.0


In [6]:
complete_motion_df.shape

(2999, 21)

At this point, we have a single dataframe with all bodies and all positions with each time step as the index of our rows.

### Checking for any NA values that could be messing up optimizer calculations in Tensorflow

In [7]:
complete_motion_df.isnull().any()
# Doesn't look like any null or missing values.

sun_x        False
sun_y        False
sun_z        False
earth_x      False
earth_y      False
earth_z      False
mars_x       False
mars_y       False
mars_z       False
venus_x      False
venus_y      False
venus_z      False
mercury_x    False
mercury_y    False
mercury_z    False
jupiter_x    False
jupiter_y    False
jupiter_z    False
saturn_x     False
saturn_y     False
saturn_z     False
dtype: bool

# Try a Quick Neural Net for Predicting Jupiter's Position

Basic Regression with Tensorflow: https://www.tensorflow.org/tutorials/keras/regression


### Imports

In [8]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
# Probably not needed since not using regressor or doing any feature engineering.
import sklearn
from sklearn.preprocessing import StandardScaler  # Scaler for normalizing data.
from sklearn.preprocessing import MinMaxScaler  # Scaler for normalizing data.
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"
# Recommended to enable eager execution when developing model.
# Processing data: https://www.youtube.com/watch?v=oFFbKogYdfc
# tf.enable_eager_execution()

# Import Keras
from tensorflow import keras

# to make this notebook's output stable across runs
#np.random.seed(42)

# Use sklearn for data processing


# Common imports
import numpy as np
import os

In [9]:
tf.__version__

'2.2.0'

In [10]:
keras.__version__

'2.3.0-tf'

## Start Here with Trying to Process Data with Tensorflow Datasets

One of the difficulties is using a mixture of numpy, pandas, and sklearn to take input data (influx), arrange it, split it out, normalize it, and then train a model.  With tf.data input pipelines (similar to sklearn pipelines), we can create data and machine learning pipelines for training or inference.  This allows us to encapsulate not only the machine learning into a Tensorflow model, but the necessary transformations to that data.  That allows us to deploy the model and data transformations as a single object to the later simulator. \
The input pipeline let's us take raw data from any source, like csv, numpy arrays, distributed file system, etc, and convert it into the tensors we will use to train our model.

In [11]:
# Got distracted and trying to figure out when and where to scale or standardize the data.

## Randomize Data, Normalize Data, and Convert Pandas Dataframe to Numpy Arrays for Input to NN

Randomize rows in dataframe before training, validation, and test splitting of data.  Shuffle dataframe in place and reset index.\ 
https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

In [12]:
complete_motion_df = complete_motion_df.sample(frac=1).reset_index(drop=True)
complete_motion_df.head(10)

,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,mars_x,mars_y,mars_z,venus_x,...,venus_z,mercury_x,mercury_y,mercury_z,jupiter_x,jupiter_y,jupiter_z,saturn_x,saturn_y,saturn_z
0,1.387984e+09,2.978031e+09,0.0,8.806298e+10,-1.232667e+11,0.0,2.628157e+10,-1.942947e+11,0.0,-9.432936e+10,...,0.0,4.964890e+10,-1.786103e+10,0.0,7.311468e+11,1.863485e+11,0.0,6.613151e+11,1.195366e+12,0.0
1,2.209324e+10,3.148353e+09,0.0,-1.124320e+11,-6.875633e+10,0.0,1.943352e+11,-1.047542e+11,0.0,-6.703942e+10,...,0.0,-2.442354e+09,5.423513e+10,0.0,-6.880208e+11,2.569786e+11,0.0,8.524204e+11,-6.424567e+11,0.0
2,5.867144e+07,3.709937e+08,0.0,-1.456760e+11,4.088025e+10,0.0,1.196600e+10,-1.979167e+11,0.0,1.098798e+11,...,0.0,-5.352711e+10,9.021154e+09,0.0,3.222454e+11,6.973305e+11,0.0,2.292013e+11,1.377177e+12,0.0
3,1.154375e+10,7.718054e+09,0.0,4.236022e+10,-1.425579e+11,0.0,1.665641e+11,-1.221680e+11,0.0,-9.852510e+10,...,0.0,-4.000508e+10,2.646141e+10,0.0,6.107699e+09,-7.169259e+11,0.0,1.171163e+12,4.110606e+11,0.0
4,1.319243e+10,7.652636e+09,0.0,-1.122565e+11,9.128067e+10,0.0,-1.226479e+11,-1.408375e+11,0.0,1.077908e+11,...,0.0,-1.382248e+10,-3.596522e+10,0.0,-1.591185e+11,-6.967806e+11,0.0,1.187410e+12,2.851910e+11,0.0
5,1.492454e+09,3.110842e+09,0.0,3.476988e+10,-1.466409e+11,0.0,-2.658184e+10,-1.937361e+11,0.0,-4.280643e+10,...,0.0,-3.809076e+10,-3.042927e+10,0.0,7.365136e+11,1.603813e+11,0.0,6.766160e+11,1.184798e+12,0.0
6,2.214689e+10,3.081993e+09,0.0,-1.242175e+11,-3.863534e+10,0.0,1.766015e+11,-1.272845e+11,0.0,-3.925814e+10,...,0.0,4.744230e+10,5.374779e+10,0.0,-6.833203e+11,2.707548e+11,0.0,8.432368e+11,-6.517450e+11,0.0
7,1.262741e+10,7.696038e+09,0.0,-1.352885e+11,-2.788923e+10,0.0,-2.407069e+10,-1.877446e+11,0.0,9.297496e+10,...,0.0,6.836932e+08,6.335658e+10,0.0,-1.028147e+11,-7.079400e+11,0.0,1.183277e+12,3.289422e+11,0.0
8,4.062784e+09,5.437939e+09,0.0,-3.699819e+10,-1.423506e+11,0.0,1.929176e+11,1.059066e+11,0.0,-8.904484e+10,...,0.0,-4.923415e+10,6.079888e+08,0.0,6.791792e+11,-2.923663e+11,0.0,9.159766e+11,9.625053e+11,0.0
9,4.876268e+09,5.936533e+09,0.0,-1.401139e+11,4.883338e+10,0.0,1.833168e+11,-9.251392e+10,0.0,5.057826e+10,...,0.0,-1.950981e+10,5.709948e+10,0.0,6.248912e+11,-3.886578e+11,0.0,9.640313e+11,8.998016e+11,0.0


Use sklearn standard scaler to normalize all columns of the dataframe.  Could also provide numpy ndarrays as input, but most likely will later need to move to normalization methods supported by Tensorflow pipelines.\
https://stackoverflow.com/questions/24645153/pandas-dataframe-columns-scaling-with-sklearn \
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html \
https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num \

This is a pretty horrible way to do it since the fit_transform object returns a numpy ndarray.  Essentially converting from dataframe, to ndarray, and back to dataframe so I can be lazy and not rewrite the code to split out the data.  I apologize in advance for being a lazy bastard.

In [ ]:
# Create Numpy ndarrays with scaled data from dataframe.  Indexes and column labels don't survive.
scaled_data = MinMaxScaler().fit_transform(complete_motion_df.values)
# Overwrite previous dataframe with scaled data and carry indexes and column labels over.
complete_motion_df = pd.DataFrame(scaled_data, index=complete_motion_df.index, columns=complete_motion_df.columns)
complete_motion_df.head(100)

In [ ]:
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
complete_motion_df.drop(complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
target.head(5)

Split the x, y, and z coordinates out for the target to use a specific dataset for each possible coordinate output.

In [ ]:
target_x = target.iloc[:,0]
target_y = target.iloc[:,1]
target_z = target.iloc[:,2]

Convert all pandas dataframes to numpy arrays.

In [ ]:
complete_motion_np = complete_motion_df.to_numpy()
target_np = target.to_numpy()
target_x_np = target_x.to_numpy()
target_y_np = target_y.to_numpy()
target_z_np = target_z.to_numpy()

Split into train, validation, and test datasets.

In [ ]:
#Split into train, validation, and test sets.
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

X_train, X_valid, X_test = complete_motion_np[:train_size], complete_motion_np[train_size:(train_size+val_size)], complete_motion_np[(train_size + val_size):]
y_train_x, y_valid_x, y_test_x = target_x_np[:train_size], target_x_np[train_size:(train_size+val_size)], target_x_np[(train_size + val_size):]
y_train_y, y_valid_y, y_test_y = target_y_np[:train_size], target_y_np[train_size:(train_size+val_size)], target_y_np[(train_size + val_size):]
y_train_z, y_valid_z, y_test_z = target_z_np[:train_size], target_z_np[train_size:(train_size+val_size)], target_z_np[(train_size + val_size):]

In [ ]:
#Block to output and troubleshoot datatypes.  For some reason, getting nan with model loss calculation.
type(X_train[0, 0])
# y_train_x

## Try Creating Single Input, Multiple Output Regression Model

Trying to create a regression NN where instead of designating an output layer of 3 nodes, 3 output layers of a single node are used to designate specific datasets and loss functions.  Still need to figure out later how to get a 3 node output to correspond to the input training data.

Use functional API to build basic NN architecture.

In [ ]:
# Use functional API to build basic NN architecture.
input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
hidden1 = keras.layers.Dense(300, activation="tanh")(input_main)
hidden2 = keras.layers.Dense(300, activation="tanh")(hidden1)
output_x = keras.layers.Dense(1, name="output_x")(hidden2)
output_y = keras.layers.Dense(1, name="output_y")(hidden2)
output_z = keras.layers.Dense(1, name="output_z")(hidden2)

Create model with specified input and output layers

In [ ]:
# Create model with specified input and output layers
model = keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])
model.summary()

In [ ]:
# Compile model with specified loss functions for each output and specify weighting to provide each output.
# Weighting X and Y output more than Z
model.compile(loss=["mse", "mse", "mse"], 
              loss_weights=[0.4, 0.4, 0.2], 
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=["mse"])

Train the model with separate x, y, z training sets.

In [ ]:
history = model.fit(
    [X_train], [y_train_x, y_train_y, y_train_z],
    epochs=1000,
    validation_data=([X_valid], [y_valid_x, y_valid_y, y_valid_z])
)

In [ ]:
# Convert training history to dataframe for analysis and plotting.
complete_history_data = pd.DataFrame(history.history)
complete_history_data.head(-9)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate MSEs.
fig2, mse_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
mse_plots[0][0].plot(complete_history_data[["output_x_mse", "val_output_x_mse"]])
#mse_plots[0][0].set_ylim(0,1)

mse_plots[0][1].plot(complete_history_data[["output_y_mse", "val_output_y_mse"]])
mse_plots[0][1].set_ylim(0,1)

mse_plots[1][0].plot(complete_history_data[["output_z_mse", "val_output_z_mse"]])
mse_plots[1][0].set_ylim(0,1)

plt.show()


In [ ]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate loss.
fig, loss_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
loss_plots[0][0].plot(complete_history_data[["loss", "val_loss"]])
loss_plots[0][0].set_ylim(0,1)

loss_plots[0][1].plot(complete_history_data[["output_x_loss", "val_output_x_loss"]])
loss_plots[0][1].set_ylim(0,1)

loss_plots[1][0].plot(complete_history_data[["output_y_loss", "val_output_y_loss"]])
loss_plots[1][0].set_ylim(0,1)

loss_plots[1][1].plot(complete_history_data[["output_z_loss", "val_output_z_loss"]])
loss_plots[1][1].set_ylim(0,1)


plt.show()


### Evaluate the Model with Test Data

In [ ]:
y_test_x.shape

In [ ]:
model.evaluate([X_test],[y_test_x, y_test_y, y_test_z])

### Predict Values and Inspect Differences

In [ ]:
x_pred, y_pred, z_pred = model.predict([X_test])

In [ ]:
pred_model_comparison = pd.DataFrame(data=np.concatenate((x_pred, y_test_x.reshape(-1,1), y_pred, y_test_y.reshape(-1,1), z_pred, y_test_z.reshape(-1,1)), axis=1),
                                    columns=['pred_x', 'model_x', 'pred_y', 'model_y', 'pred_z', 'model_z'])
pred_model_comparison.head(10)

In [ ]:
pred_model_comparison[["pred_x", "model_x"]].plot()

In [ ]:
pred_model_comparison[["pred_y", "model_y"]].plot()

In [ ]:
pred_model_comparison[["pred_z", "model_z"]].plot()